Librairies

In [1]:
import s3fs
from pathlib import Path
import py7zr
import numpy as np
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

## Données Pleiades

In [2]:
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://minio.lab.sspcloud.fr'})
fs.get('projet-slums-detection/Donnees/Cayenne_200722.7z', 'Cayenne.7z')

with py7zr.SevenZipFile('Cayenne.7z', mode='r') as z:
    z.extractall()

- Ce sont des images de 2000x2000 pixels à 50 cm le pixel => 1000x1000 m^2 = 1km^2
- On en a 240 dans le fichier brut or la superficie de Cayenne est égale à 23,6 km^2
- 4 channels RGB Infra
- Le fichier pour Cayenne fait 2 GB

## Rasterio 
https://geohackweek.github.io/raster/04-workingwithrasters/

In [3]:
import rasterio
import rasterio.plot as rp
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.plot import show
from rasterio.crs import CRS
import matplotlib
from rasterio.features import rasterize

In [4]:
im_dir = "Cayenne_200722/16bits/ORT_2022072050325085_U22N/"
liste  = os.listdir(im_dir)
list_path_image = [im_dir+l for l in liste]

len(liste)

In [5]:
## Caractéristique d'une image
filepath = list_path_image[200] 
filepath = 'Cayenne_200722/16bits/ORT_2022072050325085_U22N/ORT_2022072050325085_0353_0545_U22N_16Bits.jp2'
# 72 et 37 bons exemples
with rasterio.open(filepath) as raster:
    print(raster.profile)
    print(raster.overviews(1))
    
# En se servant du nom on peut faire des mosaiques en filtrant sur les chiffres

In [6]:
# The grid of raster values can be accessed as a numpy array and plotted:
with rasterio.open(filepath) as raster:
    oviews = raster.overviews(1) # list of overviews from biggest to smallest
    print(oviews)
    oview = 1 # let's look at the smallest thumbnail

    # NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
    B1 = raster.read(1, out_shape=(1, int(raster.height // oview), int(raster.width // oview)))
    B2 = raster.read(2, out_shape=(1, int(raster.height // oview), int(raster.width // oview)))
    B3 = raster.read(3, out_shape=(1, int(raster.height // oview), int(raster.width // oview)))
    B4 = raster.read(4, out_shape=(1, int(raster.height // oview), int(raster.width // oview)))

quantile = 0.97
B1a = rp.adjust_band(np.clip(B1,0,np.quantile(B1,quantile))) # normalisation min-max simple nécessite un clip d'abord étant donné les valeurs extremes
B2a = rp.adjust_band(np.clip(B2,0,np.quantile(B2,quantile)))
B3a = rp.adjust_band(np.clip(B3,0,np.quantile(B3,quantile)))
B4a = rp.adjust_band(np.clip(B4,0,np.quantile(B4,quantile)))

rgb = np.dstack((B1a,B2a,B3a))

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(rgb)
plt.title("Représentation de l'image {}".format(filepath))
plt.xlabel('Pixels Colonnes')
plt.ylabel('Pixels Lignes')


Fonctions clip + normalisation :

In [25]:
def clipQuant(array,quant):
    out = rp.adjust_band(np.clip(array,0,np.quantile(array,quant)))
    return(out)

def climNormStack(raster):
    oview = 1
    B1 = raster.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))
    B2 = raster.read(2, out_shape=(1, int(src.height // oview), int(src.width // oview)))
    B3 = raster.read(3, out_shape=(1, int(src.height // oview), int(src.width // oview)))
    B4 = raster.read(4, out_shape=(1, int(src.height // oview), int(src.width // oview)))

    B1a = clipQuant(B1,0.97) # normalisation min-max simple nécessite un clip d'abord étant donné les valeurs extremes
    B2a = clipQuant(B2,0.97)
    B3a = clipQuant(B3,0.97)
    B4a = clipQuant(B4,0.97)

    rgb = np.dstack((B1a,B2a,B3a))
    
    return(rgb)

## Récupération de la BDTOPO

In [9]:
import s3fs
import yaml

with open("../environment.yml", 'r') as stream:
    environment = yaml.safe_load(stream)
file_path = "projet-slums-detection/Donnees/BDTOPO/" + environment["sources"]["bdtopo"][2022]["guyane"]

fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://minio.lab.sspcloud.fr'})
fs.get(file_path, "bdtopo.7z")
with py7zr.SevenZipFile('bdtopo.7z', mode='r') as z:
    z.extractall()

In [10]:
import geopandas as gpd

bdtopo_guyane = gpd.read_file("BDTOPO_3-0_TOUSTHEMES_SHP_UTM22RGFG95_D973_2022-03-15/BDTOPO/1_DONNEES_LIVRAISON_2022-03-00081/BDT_3-0_SHP_UTM22RGFG95_D973-ED2022-03-15/BATI/BATIMENT.shp")
bdtopo_guyane.head()

In [11]:
bdtopo_guyane.info()

In [12]:
bdtopo_guyane.NATURE.unique()

In [13]:
bdtopo_guyane.USAGE1.unique()

In [14]:
bdtopo_guyane.SOURCE.unique()

Ministère en charge de la Santé et des Sports, Référentiel Géographique Guyanais sont les sources principales

Colonnes qui nous intéressent :
- NATURE
- USAGE1
- DAT_CREAT
- DAT_MAJ
- SOURCE
- GEOMETRY

In [20]:
bdtopo_guyane = bdtopo_guyane[["NATURE", "USAGE1", "DATE_CREAT", "DATE_MAJ", "SOURCE", "geometry"]]

In [21]:
bdtopo_guyane.crs

On est dans le même système de coordonnées que pour nos rasters.

### Superposition points et rasters

In [23]:
raster = rasterio.open(filepath)
xmin, ymin, xmax, ymax = raster.bounds

bdtopo_patch = bdtopo_guyane.cx[xmin:xmax,ymin:ymax]
bdtopo_patch

In [34]:
color_dict = {'Indifférenciée': 'r', 'Industriel, agricole ou commercial': 'g', 'Eglise': 'y'}

In [35]:
bdtopo_patch["color"] = bdtopo_patch["NATURE"].apply(lambda x: color_dict[x])

In [37]:
fig, ax = plt.subplots(figsize=(10, 10))
show(
    np.moveaxis(climNormStack(raster), -1, 0),
    extent = [xmin, xmax, ymin, ymax],  # sans ce paramètre c'est infaisable.. cf https://stackoverflow.com/questions/61980063/python-how-to-place-a-shapefile-on-top-of-raster-file-in-one-plot-and-then-sa
    ax=ax
)

bdtopo_patch.plot(color=bdtopo_patch["color"], ax=ax)

## Rasterisation

In [38]:
raster.shape

In [40]:
rasterized = rasterize(
    bdtopo_patch.geometry,
    out_shape = raster.shape,
    fill = 0,
    out = None,
    transform = raster.transform,
    all_touched = True,
    default_value = 1,
    dtype = None
)

# Plot raster
fig, ax = plt.subplots(1, figsize = (5, 5))
show(rasterized, ax = ax)
plt.gca().invert_yaxis()

On veut superposer les 2 rasters

In [41]:
rasterized.shape

In [44]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(rgb)
ax.imshow(rasterized, alpha=0.3)

## LIEUX-NOMMES

In [45]:
bdtopo_guyane_zh  = gpd.read_file("BDTOPO_3-0_TOUSTHEMES_SHP_UTM22RGFG95_D973_2022-03-15/BDTOPO/1_DONNEES_LIVRAISON_2022-03-00081/BDT_3-0_SHP_UTM22RGFG95_D973-ED2022-03-15/LIEUX_NOMMES/ZONE_D_HABITATION.shp")
bdtopo_guyane_zh.head()

In [46]:
bdtopo_guyane_zh.NATURE.unique()

In [47]:
bdtopo_guyane_zh = bdtopo_guyane_zh[["NATURE", "DATE_CREAT", "DATE_MAJ", "geometry"]]

## Rasterisation

In [48]:
bdtopo_guyane_zh_patch = bdtopo_guyane_zh.cx[xmin:xmax,ymin:ymax]
bdtopo_guyane_zh_patch

In [49]:
color_dict = {'Lieu-dit habité': 'r', 'Quartier': 'g'}

In [50]:
bdtopo_guyane_zh_patch["color"] = bdtopo_guyane_zh_patch["NATURE"].apply(lambda x: color_dict[x])

In [52]:
rasterized = rasterize(
    bdtopo_guyane_zh_patch.geometry,
    out_shape = raster.shape,
    fill = 0,
    out = None,
    transform = raster.transform,
    all_touched = True,
    default_value = 1,
    dtype = None
)

# Plot raster
fig, ax = plt.subplots(1, figsize = (5, 5))
show(rasterized, ax = ax)
plt.gca().invert_yaxis()

On veut superposer les 2 rasters

In [53]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(rgb)
ax.imshow(rasterized, alpha=0.3)

Ok il faudrait voir ce que ça donne les "habitats temporaires".

### Dates de MAJ, etc.

Différentes dates : 

- DATE_CONF : La Date de confirmation peut être antérieur à la Date de création de l'objet dans la base.
la Date de création est une date informatique.
Lorsqu'un bâtiment est découpé en deux, il existait "dans la réalité à une Date de confirmation T1, mais le bâtiment
issu de la découpe a une Date de création "informatique" T2 > T1.
C'est notamment le cas suite au processus d'unification des bâtiments de la BD TOPO® et du Cadastre.
- DATE_APP : Date de création, de construction ou d'apparition de l'objet, ou date la plus ancienne à laquelle on peut
attester de sa présence sur le terrain.
- DATE_CREAT : Date et heure à laquelle l'objet a été saisi pour la première fois dans la base de données de production
de l'IGN. Remarque : Il s'agit d'une date informatique. Elle n'a pas de rapport avec la date d'apparition réelle de l'objet sur le
terrain. Une scission d'un objet linéaire entraîne une création d'objet.
- DATE_MAJ : Date et heure à laquelle l'objet a été modifié pour la dernière fois dans la base de données de
production. Remarque : Il s'agit d'une date informatique. Elle peut changer lors d'une intervention sur des champs non diffusés
liés à la chaîne de production interne (commentaire non diffusé, mise à niveau...)

Finalement c'est plutôt DATE_APP qui nous intéresse. Quelque soit la version de la BDTOPO qu'on utilise on peut déjà retirer tous les bâtiments avec une DATE_APP plus tardive que la date des images satellites. PB : pas de valeurs pour cette variable. La variable renseignée est DATE_CREAT.

In [73]:
bdtopo_guyane = gpd.read_file("BDTOPO_3-0_TOUSTHEMES_SHP_UTM22RGFG95_D973_2022-03-15/BDTOPO/1_DONNEES_LIVRAISON_2022-03-00081/BDT_3-0_SHP_UTM22RGFG95_D973-ED2022-03-15/BATI/BATIMENT.shp")
bdtopo_guyane.columns

In [74]:
bdtopo_guyane.dtypes

In [75]:
bdtopo_guyane.DATE_CREAT = pd.to_datetime(bdtopo_guyane.DATE_CREAT)
bdtopo_guyane.DATE_CREAT.head()

In [77]:
bdtopo_guyane['year'] = bdtopo_guyane['DATE_CREAT'].dt.year
bdtopo_guyane['year'].unique()

In [89]:
import random

burn_dict = {year: color for year, color in zip(
    bdtopo_guyane['year'].unique(),
    [i for i in range(1, 15)]
)}

bdtopo_guyane['burn_value'] = bdtopo_guyane['year'].apply(lambda x: burn_dict[x])
bdtopo_guyane.head()

In [91]:
rasterized = rasterize(
    [(geom, burn_value) for geom, burn_value in zip(bdtopo_guyane.geometry, bdtopo_guyane.burn_value)],
    out_shape = raster.shape,
    fill = 0,
    out = None,
    transform = raster.transform,
    all_touched = True,
    default_value = 1,
    dtype = None
)

# Plot raster
fig, ax = plt.subplots(1, figsize = (5, 5))
show(rasterized, ax = ax)
plt.gca().invert_yaxis()

On veut superposer les 2 rasters

In [92]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(rgb)
ax.imshow(rasterized, alpha=0.3)

Il faudra déterminer quels bâtiments ne pas inclure dans les annotations

## Bounding boxes

On a pour le moment soit un array 0, 1 ou 0, ..., n pour faire de la segmentation, soit des géométries mais en coordonnées GPS, il nous faudrait des géométries dans le système de coordonnées de l'array.

In [94]:
rasterized = rasterize(
    bdtopo_patch.geometry,
    out_shape = raster.shape,
    fill = 0,
    out = None,
    transform = raster.transform,
    all_touched = True,
    default_value = 1,
    dtype = None
)
rasterized.shape

In [96]:
shapes = list(rasterio.features.shapes(rasterized))

In [99]:
rgb.shape

In [105]:
from shapely import Polygon

In [111]:
shape = shapes[0]

In [112]:
shape[0]["coordinates"]

In [113]:
Polygon(shape[0]["coordinates"][0])

In [157]:
from geopandas import GeoSeries
polygon_list = []
for i, shape in enumerate(shapes):
    polygon_list.append(Polygon(shape[0]["coordinates"][0]))
    if i > 658:
        break
g = GeoSeries(polygon_list)

In [158]:
len(g)

In [159]:
bdtopo_patch.shape

In [160]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(rgb)
g.plot(color="red", ax=ax)

With function `clip` :

In [162]:
bdtopo_guyane  = gpd.read_file("BDTOPO_3-0_TOUSTHEMES_SHP_UTM22RGFG95_D973_2022-03-15/BDTOPO/1_DONNEES_LIVRAISON_2022-03-00081/BDT_3-0_SHP_UTM22RGFG95_D973-ED2022-03-15/BATI/BATIMENT.shp")

bdtopo_patch_2 = gpd.clip(bdtopo_guyane, (xmin, ymin, xmax, ymax))

In [163]:
bdtopo_patch_2.head()

In [164]:
rasterized = rasterize(
    bdtopo_patch_2.geometry,
    out_shape = raster.shape,
    fill = 0,
    out = None,
    transform = raster.transform,
    all_touched = True,
    default_value = 1,
    dtype = None
)
rasterized.shape

In [165]:
shapes = list(rasterio.features.shapes(rasterized))

In [171]:
polygon_list = []
for i, shape in enumerate(shapes):
    polygon_list.append(Polygon(shape[0]["coordinates"][0]))

g = GeoSeries(polygon_list)

In [197]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(rgb)
g[:500].plot(color="red", ax=ax)

Si on essaye de clip après ?

In [175]:
rasterized = rasterize(
    bdtopo_patch.geometry,
    out_shape = raster.shape,
    fill = 0,
    out = None,
    transform = raster.transform,
    all_touched = True,
    default_value = 1,
    dtype = None
)
rasterized.shape

In [176]:
shapes = list(rasterio.features.shapes(rasterized))

In [177]:
polygon_list = []
for i, shape in enumerate(shapes):
    polygon_list.append(Polygon(shape[0]["coordinates"][0]))
        
g = GeoSeries(polygon_list)

In [180]:
g2 = gpd.clip(g, (0, 0, 2000, 2000))

In [196]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(rgb)
g2[:330].plot(color="red", ax=ax)

Comportement étrange, on veut peut-être juste éviter les aires trop grandes ?

## BB à partir de polygons

In [199]:
g2[0].bounds

Pytorch veut exactement ce format.